In [ ]:
import pandas as pd

df21 = pd.read_csv("/opt/conda/anaconda/lib/python3.6/site-packages/irsx/CSV/index_2021.csv",index_col=False)
object_ids = df21["OBJECT_ID"].to_numpy()

In [ ]:
object_ids_s = object_ids[:100]
object_ids_s

In [ ]:
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
list_all = []
for e in object_ids_s:
    filing = xml_runner.run_filing(e)
    schedules = filing.list_schedules()
    
    ein = 0
    state = 0
    name = 0
    revenue = 0
    revenueEZ = 0
    
    if "ReturnHeader990x" in schedules:
        header = filing.get_parsed_sked("ReturnHeader990x")
        #print(len(header))
        header_part_i = header[0]["schedule_parts"]["returnheader990x_part_i"]
        #print(header_part_i)
        ein = header_part_i["ein"]
        try:
            state = header_part_i["USAddrss_SttAbbrvtnCd"]
        except KeyError:
            state = XX
        name = header_part_i["BsnssNm_BsnssNmLn1Txt"]
        #print(ein, state, name)
        
    if "IRS990EZ" in schedules:
        irs990ez = filing.get_parsed_sked("IRS990EZ")
        irs990ez_part_i = irs990ez[0]["schedule_parts"]["ez_part_i"]
        revenueEZ = irs990ez_part_i["TtlRvnAmt"]        
    
    if "IRS990" in schedules:
        irs990 = filing.get_parsed_sked("IRS990")
        #print(len(irs990))
        irs990_part_i = irs990[0]["schedule_parts"]["part_i"]
        revenue = irs990_part_i["CYTtlRvnAmt"]
        #print(revenue)
        
    list = [e, ein, state, name, revenue, revenueEZ]
    list_all.append(list)
    
df_result = pd.DataFrame(list_all, columns =['ObjectID', 'EIN', 'State', 'Name', 'Revenue', 'RevenueEZ'])
df_result.to_csv('revenue_2021_100.csv', index = False)
df_result
  

        
    #print(e, ein, state, name, revenue, revenueEZ)

## Start a Sparkession

In [ ]:
from pyspark.sql.session import SparkSession
sc = SparkContext(appName="BigDataIRS3")
spark = SparkSession(sc)

## Einlesen des Files  
Das File kann entweder lokal oder mittels hdfs eingelesen werden  
Falls das File lokal eingelesen wird muss es auf jedem Node vorhanden sein, deswegen empfiehlt sich die Verwendung von HDFS 

In [ ]:
from pyspark.sql.session import SparkSession
spark2 = SparkSession.builder.appName("Test1").getOrCreate()

In [ ]:
fs_path = "file:///revenue_2021_100.csv" #local_fs
hdfs_path = "hdfs://spark-jupyter-m/user/hdfs/spark_csv/revenue_2021_100.csv" # hdfs
df = spark2.read.csv(fs_path, header=True)

type(df), df.printSchema(), df.head(5)

## Simple Aggregation
Das DF kann entweder direkt aggregiert werden, oder vorher in ein RDD umgewandelt werden

In [ ]:
!less spark_csv/revenue_2021_100.csv

In [ ]:
# Aggregation als DF
grouped_df = df.groupby("State").sum("Revenue").sort("Sum(Revenue)", ascending = False)

# Aggregation als RDD
rdd = df.rdd.drop("ObjectID", "EIN", "Name", "RevenueEZ") # U
from operator import add
reduced_rdd = rdd.reduceByKey(add).sortBy(lambda x: x[1], ascending = False)
reduced_rdd.collect()

In [ ]:
#from google.cloud import storage
#client = storage.Client()
# https://console.cloud.google.com/storage/browser/[bucket-id]/
#bucket = client.get_bucket('sparkbucket02')
# Then do other things...
#blob = bucket.get_blob("revenue_2021_100.csv") #('remote/path/to/file.txt')

#df = pd.read_csv(blob.download_as_string())
#df

In [ ]:
#spark = SparkSession \
#    .builder \
#    .appName("Protob Conversion to Parquet") \
#    .config("spark.some.config.option", "some-value") \
#    .getOrCreate()\

#df = spark.read.csv('/home/hadoop/observations_temp.csv, header=True)

sudo su - hdfs  
hdfs dfsadmin -safemode leave  

hdfs dfs -mkdir spark_csv  
hdfs dfs -put /spark_csv/revenue_2021_100.csv spark_csv/revenue_2021_100.csv  
hdfs dfs -ls spark_csv  

https://stackoverflow.com/questions/42091575/pyspark-load-file-path-does-not-exist
https://stackoverflow.com/questions/33055403/how-to-navigate-directories-in-hadoop-hdfs
https://stackoverflow.com/questions/28213116/hadoop-copy-a-local-file-system-folder-to-hdfs
https://stackoverflow.com/questions/61197811/can-i-read-csv-files-from-google-storage-using-spark-in-more-than-one-executor
https://groups.google.com/g/cloud-dataproc-discuss/c/cubkWrjkk2g?pli=1
https://stackoverflow.com/questions/56448009/storing-source-file-in-google-dataproc-hdfs-vs-google-cloud-storagegoogle-bucke

hdfs dfs -put /spark_csv/revenue_2021_100.csv /user/root/spark_csv/revenue_2021_100.csv